In [ ]:
import os
from openai import OpenAI

"""
Before submitting the assignment, describe here in a few sentences what you would have built next if you spent 2 more hours on this project:

1. As a child, I loved looking at illustrations while reading a story book.
I would have loved to integrate a lightweight HuggingFace image-prompt generator
(like BLIP, Pix2Struct, or GPT-Image Captioning) for all the major scenes of
the generated story.

2. I would experiment with a small reinforcement learning loop where the judge’s
feedback is treated as a reward signal, gently nudging the storyteller to improve style,
safety, and structure over repeated generations.



"""
import os
os.environ["OPENAI_API_KEY"] = ""

# Initialize client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


def call_model(prompt: str, max_tokens: int = 1500, temperature: float = 0.6) -> str:

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=temperature,
    )
    return response.choices[0].message.content


def build_storyteller_prompt(user_request: str, tone: str = None, length: str = "medium"):

    # Tone instruction (optional)
    tone_instruction = ""
    if tone and tone != "none":
        tone_instruction = (
            f"\nPlease give the story a '{tone}' emotional tone, "
            "while keeping it gentle and safe for children."
        )

    # Length instruction
    length_map = {
        "short": "Please write 1–2 short paragraphs (80–120 words).",
        "medium": "Please write 3–4 paragraphs (150–250 words).",
        "long": "Please write 5–6 paragraphs (250–400 words)."
    }

    length_instruction = length_map.get(length, length_map["medium"])

    # Final prompt template
    return f"""
You are a KIND, GENTLE, IMAGINATIVE STORYTELLER who writes bedtime stories for children ages 5 to 10.

The user asked for this story:
    "{user_request}"

{tone_instruction}
{length_instruction}

Please write a bedtime story that:

- Is clearly suitable for ages 5–10
- Has a simple, clear story arc: Beginning → Middle → Ending
- Uses short, easy-to-understand sentences
- Has a warm, cozy, comforting tone unless a different tone is specified
- Includes a light, positive lesson (e.g., kindness, courage, curiosity, friendship)
- Avoids all violence, fear, or adult themes
- Is about the length specified above

Return ONLY the story.
"""




def generate_story(user_request: str, tone: str = None, length: str = "medium") -> str:
    prompt = build_storyteller_prompt(user_request, tone, length)
    return call_model(prompt)







In [ ]:
def build_judge_prompt(story: str) -> str:

    return f"""
You are a STRICT children's literature evaluator. You NEVER approve a story unless it is:
- perfectly gentle
- perfectly age-appropriate (5–10 yrs)
- strongly structured (clear beginning, middle, end)
- emotionally safe
- clearly moral or meaningful
- written in simple, child-friendly vocabulary

Evaluate the story below:

\"\"\"{story}\"\"\"

Your task:
1. Provide a 2–3 sentence critique explaining whether the story is fully appropriate for ages 5–10.
2. If ANYTHING could be improved (tone, pacing, vocabulary, clarity, structure, moral lesson), you MUST NOT approve the story. Instead, provide a short bullet list (2–5 bullets) explaining what to improve.
3. Only if the story is flawless, reply with exactly:
"approved - This story is safe, gentle, and well-structured for ages 5–10."

Most stories will require improvement. Do NOT hesitate to critique.

Return ONLY the approval line OR the critique + bullet list.
"""



def judge_story(story: str) -> str:
    """
    Sends the story to the improved judge agent and returns evaluation text.
    """
    prompt = build_judge_prompt(story)
    evaluation = call_model(prompt, temperature=0.2)
    return evaluation.strip()



In [ ]:
def generate_story_with_judgment(user_request: str):
    story = generate_story(user_request)
    print("\n--- Story Draft ---\n")
    print(story)

    evaluation = judge_story(story)
    print("\n--- Judge Feedback ---\n")
    print(evaluation)

    return story, evaluation


In [ ]:
def build_reviser_prompt(story: str, judge_feedback: str) -> str:
    """
    Rewrite the story based on judge feedback while keeping it gentle and age-safe.
    """
    return f"""
You are a KIND, SKILLED children's storyteller revising a story after receiving feedback
from a literary safety judge.

Original story:
\"\"\"{story}\"\"\"

Judge feedback:
\"\"\"{judge_feedback}\"\"\"

Please REWRITE the story so that it fully satisfies every improvement requested.
Rules:
- Keep it suitable for ages 5–10
- Maintain a warm, gentle bedtime tone
- Use simple, friendly vocabulary
- Keep a clear beginning → middle → ending
- Include a soft positive lesson
- Remove ANY flagged issues
- Return ONLY the revised story
"""




In [ ]:
def revise_story(story: str, judge_feedback: str) -> str:
    """
    Calls the LLM to revise the story according to the judge's feedback.
    """
    prompt = build_reviser_prompt(story, judge_feedback)
    revised = call_model(prompt, temperature=0.5)
    return revised


In [ ]:
def generate_final_story(user_request: str, tone: str = None, length: str = "medium"):

    """
    Clean final pipeline:
    1. Story draft
    2. Judge feedback
    3. Final story (approved or revised)
    4. Moral
    """

    # 1 — STORY DRAFT
    print("\n--- 📝 STORYTELLER DRAFT ---\n")
    story = generate_story(user_request, tone, length)
    print(story)

    # 2 — JUDGE FEEDBACK
    print("\n--- 🧠 JUDGE FEEDBACK ---\n")
    feedback = judge_story(story)
    print(feedback)

    clean_fb = feedback.strip().lower()

    # 3 — FINAL STORY (approved or revised)
    if "approved" in clean_fb:
        final_story = story
    else:
        print("\n--- 🔧 REVISING STORY BASED ON FEEDBACK ---\n")
        final_story = revise_story(story, feedback)

    print("\n--- ✅ FINAL STORY ---\n")
    print(final_story)

    # 4 — MORAL
    moral_prompt = f"""
Read the story below and provide a simple, child-friendly moral (1 short sentence).

Story:
\"\"\"{final_story}\"\"\"

Return ONLY the moral.
"""
    moral = call_model(moral_prompt, temperature=0.5)

    print("\n--- 🌟 MORAL OF THE STORY ---\n")
    print(moral.strip())

    return final_story


In [ ]:
def main():
    print("\n✨ Bedtime Story System ✨")
    user_input = input("What kind of story would you like to hear today? \n> ")
    print("\nOptional: Choose an emotional tone for your story.")
    print("Options: cozy, magical, silly, sleepy, exciting, none")
    tone = input("Enter a tone or type 'none': ").strip().lower()

    print("\nChoose a story length.")
    print("Options: short, medium, long")
    length = input("Enter a length: ").strip().lower()



    final_story = generate_final_story(user_input, tone,length)




In [ ]:
if __name__ == "__main__":
    main()



✨ Hippocratic AI Bedtime Story System ✨
What kind of story would you like to hear today? 
> unicorns playing with rabbits

Optional: Choose an emotional tone for your story.
Options: cozy, magical, silly, sleepy, exciting, none
Enter a tone or type 'none': cozy

Choose a story length.
Options: short, medium, long
Enter a length: short 

--- 📝 STORYTELLER DRAFT ---

Once upon a time, in a magical forest filled with colorful flowers and sparkling streams, there lived a group of unicorns and rabbits who were the best of friends. Every day, the unicorns would use their shimmering horns to create rainbows for the rabbits to play on, while the rabbits would hop around, giggling with joy.

One sunny afternoon, the unicorns and rabbits decided to have a playful race through the meadow. The unicorns galloped gracefully, their hooves barely touching the ground, while the rabbits hopped swiftly, their fluffy tails bouncing behind them. In the end, they all crossed the finish line together, laugh